# Stock Price Forecasting AI Agent

This project is an AI-powered function-calling agent designed to forecast stock prices based on user-defined parameters. By leveraging the OpenAI API and advanced neural forecasting models, this agent enables users to receive accurate stock predictions for specified periods. The project is tailored for financial analysts, traders, and developers seeking quick, AI-driven insights into stock trends.

## Features

- **Customizable Forecast Period**: Users can specify the number of days (`prediction_days`) for the stock forecast, providing flexibility based on individual needs.
- **Function Calling with OpenAI API**: The agent uses OpenAI's function calling capabilities to interpret user prompts, extract relevant details like `stock_symbol` and `prediction_days`, and trigger the forecast function.
- **Advanced Forecasting Model**: Integrates the Forecasting LLM model Chronos from AWS.
- **Automatic Data Retrieval**: Pulls historical stock data from Yahoo Finance, processes it, and prepares the dataset for model training and prediction.

## Workflow

1. **User Prompt**: The user specifies a stock symbol (e.g., "AAPL" or "MSFT") and a forecast duration in days.
2. **Agent Function Calling**: The agent, powered by OpenAI API, parses the prompt to identify the stock symbol and prediction days.
3. **Data Retrieval and Preparation**: Stock data is retrieved for the past year and preprocessed for the forecasting model.
4. **Forecast Generation**: Using NBEATS, the agent generates a stock price forecast, returning a dataframe with dates, predictions, and the ticker symbol.

## Example Usage

1. **Define User Request**: "Get stock price forecast for Qualcomm for 15 days."
2. **Agent Identifies Parameters**: The function schema parses `stock_symbol` as "QCOM" and `prediction_days` as 15.
3. **Generate and Output Forecast**: The agent calls the `get_stock_forecast` function, providing a 15-day forecast.

In [1]:
import yfinance as yf
import plotly.graph_objects as go
import torch
from chronos import ChronosPipeline
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple


/Users/johtorr/Repos/stock_forecast_ai_agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import requests
import os
import json
import plotly.graph_objects as go

## Define Forecasting Function 

Use LLM Chronos Model from AWS.

In [3]:
def get_stock_forecast(stock_symbol: str, prediction_days: int = 10) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Forecasting function for any given ticker

    Args:
        stock_symbol (str): The stock symbol to forecast (e.g., 'AAPL', 'MSFT').
        prediction_days (int, optional): The prediction days. Defaults to 10.

    Returns:
        pd.DataFrame: Dataframe containing: 
            - ticker (string)
            - date (datetime.date)
            - prediction (float)
    """

    stock_data = yf.download(stock_symbol, period="1y", interval="1d")
    
    stock_data = stock_data.reset_index().rename(columns={"Date": "ds", "Close": "y"})
    stock_data = stock_data[["ds", "y"]].copy()
    stock_data.columns = ["ds", "y"]
    stock_data['unique_id'] = stock_symbol

    pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="mps", # configure Mac MPS GPU
    torch_dtype=torch.bfloat16,
    )   

    historical_data = stock_data['y'].tolist()

    context = torch.tensor(historical_data)

    # predict using LLM model by passing context data
    forecasts = pipeline.predict(context, prediction_days)  

    df_forecast = pd.DataFrame()
    for ts_key, forecast in zip([stock_symbol], forecasts):
        low, median, high = np.quantile(forecast.numpy(), [0.1, 0.5, 0.9], axis=0)

        df_forecast = pd.DataFrame({'forecast_lower': low,
                            'forecast_median':median,
                            'forecast_high':high
                            })
        df_forecast['ticker'] = ts_key

    return df_forecast, stock_data

In [4]:
# Test the function
df_forecast, stock_data = get_stock_forecast('TSLA', 20)
df_forecast.head()

[*********************100%***********************]  1 of 1 completed


,forecast_lower,forecast_median,forecast_high,ticker
0,342.450800,348.560410,367.359177,TSLA
1,333.208099,353.260086,366.419244,TSLA
2,333.521414,351.693527,365.009338,TSLA
3,327.098462,349.343689,377.698495,TSLA
4,331.954852,353.260086,388.977750,TSLA


## Create the Stock Forecasting Agent

In [7]:

def stock_forecast_agent(prompt: str)-> Tuple[pd.DataFrame, pd.DataFrame, Dict]:
    """Using OpenAI api and GPT4mini predict the stock price 
    of a company available in the Yahoo Finance API for any days in the futures.

    Args:
        prompt (str): the user prompt
    """
        
    api_key = os.environ['OPENAI_API_KEY']
    endpoint = "https://api.openai.com/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Define the function schema for OpenAI function calling
    function_schema = {
        "name": "get_stock_forecast",
        "description": "Get stock price forecast for a given number of days",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_symbol": {
                    "type": "string",
                    "description": "The stock symbol to forecast (e.g., 'AAPL', 'MSFT')."
                },
                "prediction_days": {
                    "type": "integer",
                    "description": "The number of days to forecast prices for."
                }
            },
            "required": ["stock_symbol"]
        }
    }

    messages = [
        {"role": "user", "content": prompt}
    ]

    data = {
        "model": "gpt-4o-mini",  
        "messages": messages,
        "functions": [function_schema],
        "function_call": "auto"
    }

    response = requests.post(endpoint, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        result = response.json()
        function_call = result["choices"][0].get("message", {}).get("function_call")

        if function_call:
            arguments = json.loads(function_call["arguments"])
            stock_symbol = arguments["stock_symbol"]
            prediction_days = arguments.get("prediction_days", 10)
            df_forecast, df_historical_data = get_stock_forecast(stock_symbol, prediction_days)
        else:
            print(result["choices"][0]["message"]["content"])
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")

    return df_forecast, df_historical_data, arguments


In [12]:
prompt = "Get stock price forecast for TESLA for 15 days"
forecast, df_historical_data, arguments = stock_forecast_agent(prompt=prompt)
print(forecast.head())

[*********************100%***********************]  1 of 1 completed


   forecast_lower  forecast_median  forecast_high ticker
0      334.928024       348.556992     363.595865   TSLA
1      338.217795       349.340256     365.162421   TSLA
2      328.661819       347.773712     374.091730   TSLA
3      330.228375       346.990448     370.958643   TSLA
4      325.058768       345.423904     367.668878   TSLA


## Plot Historical Data and Forecast Values

In [13]:
historical_horizon = 100
stock_symbol = arguments["stock_symbol"]
prediction_days = arguments.get("prediction_days", 10)
historical_dates = df_historical_data['ds'].values[-historical_horizon:]
historical_prices = df_historical_data['y'].values[-historical_horizon:]

forecast_dates = pd.date_range(
    start=historical_dates[-1] + pd.Timedelta(days=1), periods=prediction_days
)

fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(
    x=historical_dates,
    y=historical_prices,
    mode='lines',
    name='Historical Price',
    line=dict(color='blue')
))

# Add forecast median
fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=forecast["forecast_median"],
    mode='lines',
    name='Median Forecast',
    line=dict(color='orange')
))

# Add forecast range (low-high) as a filled area
fig.add_trace(go.Scatter(
    x=forecast_dates.tolist() + forecast_dates[::-1].tolist(),
    y=forecast["forecast_high"].tolist() + forecast["forecast_lower"][::-1].tolist(),
    fill='toself',
    name='Forecast Range (Low-High)',
    fillcolor='rgba(255, 165, 0, 0.3)',
    line=dict(color='rgba(255, 165, 0, 0)')
))

# Customize the layout
fig.update_layout(
    title=f"{stock_symbol} Historical Data and Forecast",
    xaxis_title="Date",
    yaxis_title="Price",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    template="plotly_white"
)

fig.show()